# Análisis temporal de precios

Este notebook carga `data/prices.csv` (o `data/sample_prices.csv` si no existe) y muestra visualizaciones de la evolución de precios por producto y por sitio. Usa pandas y Plotly para interactividad.

In [ ]:
# Importar librerías
import os
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime

pd.options.plotting.backend = 'matplotlib'

In [ ]:
# Cargar datos: usar data/prices.csv si existe o el sample
path = 'data/prices.csv'
if not os.path.exists(path):
    path = 'data/sample_prices.csv'

df = pd.read_csv(path, parse_dates=['timestamp'])
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df = df.dropna(subset=['price'])
df['date'] = pd.to_datetime(df['timestamp']).dt.date
df.head()

## Serie temporal por producto
Visualizamos la evolución del precio agregada por día para cada producto.

In [ ]:
# Agregar precio medio por día y producto
df_group = df.groupby(['date','product'])['price'].mean().reset_index()
fig = px.line(df_group, x='date', y='price', color='product', markers=True, title='Precio promedio diario por producto')
fig.update_layout(xaxis_title='Fecha', yaxis_title='Precio')
fig.show()

## Comparación entre sitios para un producto
Filtramos por un producto y comparamos la evolución por sitio.

In [ ]:
# Elegir producto de ejemplo (cambiar si hace falta)
producto = df['product'].unique()[0]
df_prod = df[df['product'] == producto].copy()
df_prod_group = df_prod.groupby(['date','site'])['price'].mean().reset_index()
fig2 = px.line(df_prod_group, x='date', y='price', color='site', markers=True, title=f'Evolución de {producto} por sitio')
fig2.update_layout(xaxis_title='Fecha', yaxis_title='Precio')
fig2.show()

## Media móvil y detección de cambios
Calculamos una media móvil simple para suavizar la serie e identificar tendencias.

In [ ]:
# Media móvil por producto (7 días)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df_sorted = df.sort_values('timestamp')
df_sorted['date'] = df_sorted['timestamp'].dt.date
prod = df_sorted['product'].unique()[0]
s = df_sorted[df_sorted['product'] == prod].set_index('timestamp').resample('D')['price'].mean().ffill()
ma = s.rolling(window=7, min_periods=1).mean()
plt.figure(figsize=(10,5))
plt.plot(s.index, s.values, label='Precio diario')
plt.plot(ma.index, ma.values, label='MA(7)')
plt.title(f'Precio y MA(7) para {prod}')
plt.xlabel('Fecha')
plt.ylabel('Precio')
plt.legend()
plt.grid(True)
plt.show()